In [1]:
import time
import dask.dataframe as dd
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
icuCharDD = dd.read_csv(
    "chartevents.csv",
    dtype={
        "caregiver_id": "float64",
        "value": "object",
        "valuenum": "float64",
        "warning": "float64",
    },
)

icuCharDD_sCr = icuCharDD[icuCharDD["itemid"] == 220615]
icuCharDD_sCr=icuCharDD_sCr.dropna(subset=['valuenum'])


In [3]:
print("compute")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))

icuChar_sCr = icuCharDD_sCr.compute()
icuChar_sCr=icuChar_sCr.reset_index(drop=True)

print("end compute")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))
#4min

compute
20230601_080749
end compute
20230601_081054


In [4]:
staticsCr=pd.DataFrame()
staticsCr.loc[0,'item']=220615
staticsCr.loc[0,'max']=np.max(icuChar_sCr['valuenum'])
staticsCr.loc[0,'min']=np.min(icuChar_sCr['valuenum'])
staticsCr.loc[0,'std']=np.std(icuChar_sCr['valuenum'])
staticsCr.loc[0,'average']=np.average(icuChar_sCr['valuenum'])
staticsCr.loc[0,'mean']=np.mean(icuChar_sCr['valuenum'])
staticsCr.loc[0,'median']=np.median(icuChar_sCr['valuenum'])
lowCut=np.percentile(icuChar_sCr['valuenum'],0.1)
staticsCr.loc[0,'p0.1']=lowCut
staticsCr.loc[0,'p10']=np.percentile(icuChar_sCr['valuenum'],10)
staticsCr.loc[0,'p20']=np.percentile(icuChar_sCr['valuenum'],20)
staticsCr.loc[0,'p25']=np.percentile(icuChar_sCr['valuenum'],25)
staticsCr.loc[0,'p75']=np.percentile(icuChar_sCr['valuenum'],75)
staticsCr.loc[0,'p80']=np.percentile(icuChar_sCr['valuenum'],80)
staticsCr.loc[0,'p90']=np.percentile(icuChar_sCr['valuenum'],90)
highCut=np.percentile(icuChar_sCr['valuenum'],99.9)
staticsCr.loc[0,'p99.9']=highCut

icuChar_sCr_clean=icuChar_sCr[(lowCut<=icuChar_sCr['valuenum'])&(icuChar_sCr['valuenum']<=highCut)]

staticsCr.loc[0,'max_clean']=np.max(icuChar_sCr_clean['valuenum'])
staticsCr.loc[0,'min_clean']=np.min(icuChar_sCr_clean['valuenum'])
staticsCr.loc[0,'std_clean']=np.std(icuChar_sCr_clean['valuenum'])
staticsCr.loc[0,'average_clean']=np.average(icuChar_sCr_clean['valuenum'])
staticsCr.loc[0,'mean_clean']=np.mean(icuChar_sCr_clean['valuenum'])
staticsCr.loc[0,'median_clean']=np.median(icuChar_sCr_clean['valuenum'])
staticsCr.loc[0,'p0.1_clean']=np.percentile(icuChar_sCr_clean['valuenum'],0.1)
staticsCr.loc[0,'p10_clean']=np.percentile(icuChar_sCr_clean['valuenum'],10)
staticsCr.loc[0,'p20_clean']=np.percentile(icuChar_sCr_clean['valuenum'],20)
staticsCr.loc[0,'p25_clean']=np.percentile(icuChar_sCr_clean['valuenum'],25)
staticsCr.loc[0,'p75_clean']=np.percentile(icuChar_sCr_clean['valuenum'],75)
staticsCr.loc[0,'p80_clean']=np.percentile(icuChar_sCr_clean['valuenum'],80)
staticsCr.loc[0,'p90_clean']=np.percentile(icuChar_sCr_clean['valuenum'],90)
staticsCr.loc[0,'p99.9_clean']=np.percentile(icuChar_sCr_clean['valuenum'],99.9)


In [5]:
saveTimeStr = time.strftime("%Y%m%d_%H%M%S", time.localtime())
staticsCr.to_excel(saveTimeStr+'_mimiciv_icuChar_sCr_statistical.xlsx',index=False)

In [6]:
saveTimeStr = time.strftime("%Y%m%d_%H%M%S", time.localtime())
icuChar_sCr_clean.to_csv(saveTimeStr+'_mimiciv_icuChar_sCr_clean.csv',index=False)
#2s

In [ ]:
#quick save point
icuChar_sCr_clean=pd.read_csv('???_mimiciv_icuChar_sCr_clean.csv')

In [7]:
admi=pd.read_csv("20230525_160146mimiciv_hosp_admi.csv")

In [8]:
icuChar_sCr_clean_admi = icuChar_sCr_clean.merge(admi, how="left", on=["subject_id","hadm_id"])
#

In [9]:
print("to_datetime")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))

icuChar_sCr_clean_admi["charttime"] = pd.to_datetime(icuChar_sCr_clean_admi["charttime"], format="%Y-%m-%d %H:%M:%S")
icuChar_sCr_clean_admi["admittime"] = pd.to_datetime(icuChar_sCr_clean_admi["admittime"], format="%Y-%m-%d %H:%M:%S")
icuChar_sCr_clean_admi["dischtime"] = pd.to_datetime(icuChar_sCr_clean_admi["dischtime"], format="%Y-%m-%d %H:%M:%S")
icuChar_sCr_clean_admi["deathtime"] = pd.to_datetime(icuChar_sCr_clean_admi["deathtime"], format="%Y-%m-%d %H:%M:%S")

print("end to_datetime")
print(time.strftime("%Y%m%d_%H%M%S", time.localtime()))

to_datetime
20230601_081058
end to_datetime
20230601_081058


In [10]:
icuChar_sCr_clean_admi=icuChar_sCr_clean_admi.sort_values(by=['subject_id', 'hadm_id','charttime'])
icuChar_sCr_clean_admi=icuChar_sCr_clean_admi.reset_index(drop=True)

In [11]:
icuChar_sCr_clean_admi["delta_in_char_time(hours)"] = (icuChar_sCr_clean_admi["charttime"] - icuChar_sCr_clean_admi["admittime"]) / np.timedelta64(1, "h")

icuChar_sCr_clean_admi["delta_char_disc_time(hours)"] = (icuChar_sCr_clean_admi["charttime"] - icuChar_sCr_clean_admi["dischtime"]) / np.timedelta64(1, "h")

icuChar_sCr_clean_admi["delta_char_death_time(hours)"] = (icuChar_sCr_clean_admi["charttime"] - icuChar_sCr_clean_admi["deathtime"]) / np.timedelta64(1, "h")

icuChar_sCr_clean_admi['delta_time(hours)']=icuChar_sCr_clean_admi["delta_in_char_time(hours)"]

In [12]:
icuChar_sCr_clean_admi[:5]

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,admittime,dischtime,deathtime,race,hospital_expire_flag,delta_in_char_time(hours),delta_char_disc_time(hours),delta_char_death_time(hours),delta_time(hours)
0,10000032,29079034,39553978,NaN,2180-07-23 21:45:00,2180-07-23 22:30:00,220615,0.5,0.5,mg/dL,0.0,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,WHITE,0,9.166667,-44.166667,NaN,9.166667
1,10001217,24597018,37067082,NaN,2157-11-21 03:16:00,2157-11-21 04:53:00,220615,0.4,0.4,mg/dL,0.0,2157-11-18 22:56:00,2157-11-25 18:00:00,NaT,WHITE,0,52.333333,-110.733333,NaN,52.333333
2,10001217,27703517,34592300,NaN,2157-12-20 01:45:00,2157-12-20 02:41:00,220615,0.5,0.5,mg/dL,0.0,2157-12-18 16:58:00,2157-12-24 14:55:00,NaT,WHITE,0,32.783333,-109.166667,NaN,32.783333
3,10001725,25563031,31205490,NaN,2110-04-11 18:02:00,2110-04-11 18:53:00,220615,0.8,0.8,mg/dL,0.0,2110-04-11 15:08:00,2110-04-14 15:00:00,NaT,WHITE,0,2.900000,-68.966667,NaN,2.900000
4,10001725,25563031,31205490,NaN,2110-04-12 02:59:00,2110-04-12 03:47:00,220615,0.8,0.8,mg/dL,0.0,2110-04-11 15:08:00,2110-04-14 15:00:00,NaT,WHITE,0,11.850000,-60.016667,NaN,11.850000


In [13]:
icuChar_sCr_clean_admi_inTime=icuChar_sCr_clean_admi[(0<icuChar_sCr_clean_admi["delta_in_char_time(hours)"])
                                            &(icuChar_sCr_clean_admi["delta_in_char_time(hours)"]<=168)
                                            &(icuChar_sCr_clean_admi["delta_char_disc_time(hours)"]<=0)]

In [14]:
icuChar_sCr_clean_admi_inTime=icuChar_sCr_clean_admi_inTime.sort_values(by=['subject_id', 'hadm_id'])
icuChar_sCr_clean_admi_inTime=icuChar_sCr_clean_admi_inTime.reset_index(drop=True)

In [15]:
saveTimeStr = time.strftime("%Y%m%d_%H%M%S", time.localtime())
icuChar_sCr_clean_admi_inTime.to_csv(saveTimeStr+'mimiciv_icuChar_sCr_clean_inTime.csv',index=False)
#4s

In [ ]:
#quick save point

icuChar_sCr_clean_admi_inTime=pd.read_csv('??mimiciv_icuChar_sCr_clean_inTime.csv')


icuChar_sCr_clean_admi_inTime["charttime"] = pd.to_datetime(
    icuChar_sCr_clean_admi_inTime["charttime"], format="%Y-%m-%d %H:%M:%S"
)

icuChar_sCr_clean_admi_inTime["admittime"] = pd.to_datetime(
    icuChar_sCr_clean_admi_inTime["admittime"], format="%Y-%m-%d %H:%M:%S"
)


icuChar_sCr_clean_admi_inTime["dischtime"] = pd.to_datetime(
    icuChar_sCr_clean_admi_inTime["dischtime"], format="%Y-%m-%d %H:%M:%S"
)

icuChar_sCr_clean_admi_inTime["deathtime"] = pd.to_datetime(
    icuChar_sCr_clean_admi_inTime["deathtime"], format="%Y-%m-%d %H:%M:%S"
)



In [16]:
def tKDIGO_kn(inputt):
    inputt=inputt.sort_values(by=['charttime'])
    out = pd.DataFrame()

    out.loc[0, 'subject_id'] = inputt.iloc[0]['subject_id']
    out.loc[0, 'hadm_id'] = inputt.iloc[0]['hadm_id']
    out.loc[0, 'admittime'] = inputt.iloc[0]['admittime']

    akiHappenAt = inputt.iloc[len(inputt)-1]['delta_time(hours)']

    if len(inputt) < 2:
        out.loc[0, 'AKI_happened'] = np.nan
        out.loc[0, 'AKI_time'] = np.nan
        out.loc[0, 'KDIGO_Stage'] = np.nan
        out.loc[0, 'deltCre01'] = np.nan
        out.loc[0, 'deltCre02'] = np.nan
        out.loc[0, 'delta_time(hours)'] = np.nan
        return out
    # =================================================================================================
    tmp = inputt[(inputt['delta_time(hours)'] <= 48)]
    deltCre01 = 0
    chtime01 = inputt.iloc[0]['charttime']
    if len(tmp) > 1:
        #deltCre01 = tmp['valuenum'].max()-tmp['valuenum'].min()
        for ii in range(len(tmp)-1):
            v1 = tmp.iloc[0]['valuenum']
            v2 = tmp.iloc[ii+1]['valuenum']
            v3 = v2-v1
            if v3 >= 0.3:
                ftt = tmp.iloc[ii+1]['delta_time(hours)']
                if akiHappenAt > ftt:
                    akiHappenAt = ftt
            if deltCre01 < v3:
                deltCre01 = v3
                chtime01 = tmp.iloc[ii+1]['charttime']

    # =================================================================================================
    tmp = inputt[(inputt['delta_time(hours)'] <= 168)]
    deltCre02 = 0
    chtime02 = inputt.iloc[0]['charttime']
    if len(tmp) > 1:
        for ii in range(len(tmp)-1):
            v1 = tmp.iloc[0]['valuenum']
            v2 = tmp.iloc[ii+1]['valuenum']
            v3 = (v2-v1)/v1
            if v3 >= 1.5:
                ftt = tmp.iloc[ii+1]['delta_time(hours)']
                if akiHappenAt > ftt:
                    akiHappenAt = ftt
            if deltCre02 < v3:
                deltCre02 = v3
                chtime02 = tmp.iloc[ii+1]['charttime']

    # =================================================================================================
    if deltCre01 >= 0.3 or deltCre02 >= 1.5:
        out.loc[0, 'AKI_happened'] = 1
        hapnTime = (chtime02 - chtime01) / np.timedelta64(1, 'h')
        if hapnTime >= 0:
            out.loc[0, 'AKI_time'] = chtime02
        else:
            out.loc[0, 'AKI_time'] = chtime01

        if (deltCre01 >= 4) or (3 <= deltCre02):
            out.loc[0, 'KDIGO_Stage'] = 3
        elif (2 <= deltCre02) and (deltCre02 < 3):
            out.loc[0, 'KDIGO_Stage'] = 2
        elif ((1.5 <= deltCre02) and (deltCre02 < 2)) or (0.3 <= deltCre01):
            out.loc[0, 'KDIGO_Stage'] = 1
        else:
            out.loc[0, 'KDIGO_Stage'] = -100

    else:
        out.loc[0, 'AKI_happened'] = 0
        out.loc[0, 'AKI_time'] = np.nan
        out.loc[0, 'KDIGO_Stage'] = 0

    # =================================================================================================
    out.loc[0, 'deltCre01'] = deltCre01
    out.loc[0, 'deltCre02'] = deltCre02
    out.loc[0, 'delta_time(hours)'] = akiHappenAt

    return out




In [17]:
mimiciv_icu_aki=icuChar_sCr_clean_admi_inTime.groupby(['subject_id','hadm_id']).apply(tKDIGO_kn).reset_index(drop=True)
#7min

In [18]:
mimiciv_icu_aki=mimiciv_icu_aki.dropna(subset=['AKI_happened'])
mimiciv_icu_aki=mimiciv_icu_aki.reset_index(drop=True)

In [19]:
saveTimeStr = time.strftime("%Y%m%d_%H%M%S", time.localtime())
mimiciv_icu_aki.to_csv(saveTimeStr+'mimiciv_icu_aki.csv',index=False)

In [20]:
def tSelectFrist(inDf):
    inDf=inDf.sort_values(by=['admittime'])

    fAki=inDf[inDf['AKI_happened']>0.01]
    if len(fAki)>0:
        return fAki.iloc[0]
    
    return inDf.iloc[0]

In [21]:
mimiciv_icu_aki_frist=mimiciv_icu_aki.groupby(['subject_id']).apply(tSelectFrist).reset_index(drop=True)
mimiciv_icu_aki_frist=mimiciv_icu_aki_frist.sort_values(by=['subject_id', 'hadm_id'])

mimiciv_icu_aki_frist=mimiciv_icu_aki_frist[['subject_id', 'hadm_id', 'AKI_happened', 'AKI_time','KDIGO_Stage', 'delta_time(hours)']]

mimiciv_icu_aki_frist=mimiciv_icu_aki_frist.reset_index(drop=True)
#28s

In [22]:
saveTimeStr = time.strftime("%Y%m%d_%H%M%S", time.localtime())
mimiciv_icu_aki_frist.to_csv(saveTimeStr+'mimiciv_icu_aki_frist.csv',index=False)

In [23]:
pp=pd.unique(mimiciv_icu_aki_frist['subject_id'])
print(len(pp))
#40747

40747


In [24]:
pp=pd.unique(mimiciv_icu_aki_frist['hadm_id'])
print(len(pp))
#40747

40747


In [25]:
pp=mimiciv_icu_aki_frist[mimiciv_icu_aki_frist['AKI_happened']==1]
print(len(pp))
#5749

5749
